##(Re)Intro to Neural Nets and Deep Learning

#What you'll learn today
1.  Different neural network architectures
2.  What exactly is "Deep Learning" and why the buzz?
3.  Neural Nets have been around since the 40's.  What makes them current?

#Topics for today
1.  Some standard neural net architectures that are used in practice
2.  Definition of deep network (or deep learning) and training complications
3.  Enabling developments for deep learning

#Pre-reading
Skim these articles to get a feel for what they cover and to generate some questions that you want to get answered during the lecture.  This lecture will go through portions of these in more detail. 

https://en.wikipedia.org/wiki/Feedforward_neural_network

http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf

https://en.wikipedia.org/wiki/Convolution

http://deeplearning.net/tutorial/lenet.html

https://en.wikipedia.org/wiki/Convolutional_neural_network

http://www.willamette.edu/~gorr/classes/cs449/rnn1.html

http://ufldl.stanford.edu/wiki/index.php/Stacked_Autoencoders



#Fully-connected feed forward neural networks
https://en.wikipedia.org/wiki/Feedforward_neural_network

#What is deep learning?
"Deep learning" refers to a (feedforward) network with more than one hidden layer.  Why are these special?  Deep networks have multiple layers.  Each layer takes the features it is given from the layer below and aggregates them into higher level features.  Think about the process of classifying an image.  The low level features might be edges and textures.  Think of the first layer in the network as detecting edges.  Then the next layer might aggregate the edges into higher level features like corners and curves.  The next might assemble those into distinct objects - like a body, nose, eyes, hair.  The next might assemble those into an animal and the last layer could classify the animal as cat based on the relationships of the lower level features.  Making the identification requires several layers of abstraction to reach the final result.  Recognizing spoken language is another example.  The lowest level features are individual sounds (phonemes).  Those get assembled into words, which get assembled into sentences which combine with other sentences to deliver a good story.  Deep learning has proven uniquely well-suited to solving problems of this type.  What took so long?

Deeper networks are difficult to train.  Maybe that's why it's faster to house break a dog than a child.  The technology for getting past that training complexity has only been developed in the last 10 years or so.  Later in this lecture you'll go through the Glorot, Bengio paper.  That will give you an understanding of the sources and cures for training problems in deep networks.  Luckily for those of you taking the class, these technologies are relatively new and still developing.  There are numerous opportunities for you to apply them to interesting problems.  


#Q's

1.  How many hidden layers are required to qualify as a "deep" network?

2.  How does the data from a photo get into a neural net?  Where does the numeric value into the first input layer neuron come from?  

3.  Suppose you have an input layer with 1000 input neurons (input attributes) and the first hidden layer has 500 neurons.  How many weights are there between the input layer and the first hidden layer?  

#Convolutional neural nets
A convolutional neural net (or a convolutional layer in a neural net) is a specialized type of feedforward structure. In a convolutional layer not every input to the layer is connected to every neuron in the next layer.  Only chosen subsets of the inputs are connected.  Image-based prediction is the most familiar problem where convolutional nets are frequently used.  The rationale for choosing subsets of image pixels is that adjacent pixels are often highly correlated so groups of them can be processed similarly.  But images aren't the only place where convolutional processing can play a role.  Time series data provides a one-dimensional setting where a convolutional operator can be useful.  The wiki page on convolution shows an animation of the convolution process in a time series application.  

https://en.wikipedia.org/wiki/Convolution

The animation depicts the convolution of a rectangular window with two different time-based waveforms - a square pulse and an exponential decay function.  For both of these the convolution of the waveforms with a square window smooths the edges out from the original waveform.  Convolving a different function could serve to accentuate the edges.  


#Q's

1.  Describe a function that would accentuate the edges of a time-based waveform.

With a 2-d image the convolution functions are also 2d.  

#Q's

1.  What is the dimensionality of a color image?  

2.  Characterize a convolution function for a color image.  

Convolutional layers are frequently used as part of an image process sequence.  Images can easily have millions of pixels.  Their extraordinarily high dimensionality makes training very slow.  One or more convolutional layers are used to reduce dimensionality and are frequently followed by one or more fully connected layers.  

The section on convolutional neural nets from deeplearning.net has a good summary and some interesting code samples.  

http://deeplearning.net/tutorial/lenet.html

Figure 1 gives a depiction of a convolutional layer for 4-color image data.  THe convolutional function depicted in that figure comprises a 2-by-2 patch from each of the four color planes.  Each pixel in the 4, 2-by-2 patches is weighted by a unique weight and the results are summed.  The resulting sums are depicted by the two planes shown on the right side of Figure 1.  One of these corresponds to one set of weights and the other corresponds to a different set of weights.  

The deeplearning.net page gives an interesting example of the application of a convolution function.  The code initializes a two convolution functions with random weights and then applies the two random convolution functions to the picture of three wolves shown in the example.  

In [2]:
__author__ = 'mike.bowles'
#code from http://deeplearning.net/tutorial/lenet.html
import theano
from theano import tensor as T
from theano.tensor.nnet import conv
import numpy as np
import pylab
from PIL import Image


rng = np.random.RandomState(23455)

# instantiate 4D tensor for input
input = T.tensor4(name='input')

# initialize shared variable for weights.
w_shp = (2, 3, 9, 9)
w_bound = np.sqrt(3 * 9 * 9)
W = theano.shared( np.asarray(
            rng.uniform(
                low=-1.0 / w_bound,
                high=1.0 / w_bound,
                size=w_shp),
            dtype=input.dtype), name ='W')

newW = W

#try making up your own convolution functions.
# Here's a blur function that replaces pixel values with the average over
# the 9x9 square.  uncomment the next 4 lines

#const = 1.0 / (3 * 9 * 9) #normalizing constant
#Y = theano.shared(np.ones((3,9,9)) * const)
#print Y.get_value()

#newW = T.set_subtensor(W[1,:, :, :], Y)



# initialize shared variable for bias (1D tensor) with random values
# IMPORTANT: biases are usually initialized to zero. However in this
# particular application, we simply apply the convolutional layer to
# an image without learning the parameters. We therefore initialize
# them to random values to "simulate" learning.
b_shp = (2,)
b = theano.shared(np.asarray(
            rng.uniform(low=-.5, high=.5, size=b_shp),
            dtype=input.dtype), name ='b')

# build symbolic expression that computes the convolution of input with filters in w
conv_out = conv.conv2d(input, newW)


output = T.nnet.sigmoid(conv_out + b.dimshuffle('x', 0, 'x', 'x'))

# create theano function to compute filtered images
f = theano.function([input], output)



# open random image of dimensions 639x516
img = Image.open(open('3wolfmoon.jpg'))
# dimensions are (height, width, channel)
img = np.asarray(img, dtype='float64') / 256.

# put image in 4D tensor of shape (1, 3, height, width)
img_ = img.transpose(2, 0, 1).reshape(1, 3, 639, 516)
filtered_img = f(img_)

# plot original image and first and second components of output
pylab.subplot(1, 3, 1); pylab.axis('off'); pylab.imshow(img)
pylab.gray();
# recall that the convOp output (filtered image) is actually a "minibatch",
# of size 1 here, so we take index 0 in the first dimension:
pylab.subplot(1, 3, 2); pylab.axis('off'); pylab.imshow(filtered_img[0, 0, :, :])
pylab.subplot(1, 3, 3); pylab.axis('off'); pylab.imshow(filtered_img[0, 1, :, :])
pylab.show()

It's remarkable that randomly initialized weights act so clearly like an edge detector.  There a commented section of the code that will make the second convolutional function a blur filter rather than an edge detector.  Uncomment those lines and run the code again.  

#Q's

1.  Describe the operation of the blur filter.  

2.  Compare the operation of the blur filter with the animation showing the operation of a box-shaped filter on time series data.  

3.  What would you make the weights if you were building an edge detector?  

4.  Build an edge detector and incorporate it in the code snip above.  

#Using convolutional layer in a neural net.  

The discussion above described convolutional network layers as a dimensionality reducing technique.  The convolutional filter that you just saw returns a couple of pictures with more-or-less the same number of pixels as each of the color planes in the original picture.  Not much of a dimensionality reduction.  The real dimensionality reduction with a convolutional layer comes after the application of several convolutional filters.  The filtered pixels are further processed some type of non-linearity - usually either a hyperbolic tangent or what's called a rectified linear unit.  Then comes what's called a pooling layer.  

Pooling aggregates the outputs of a block of the pixels from the convolutional filter.  The usual choices for the aggregation function are taking the maximum value or averaging the pixel values.  The pooling layer results in a reduction in the number of features because a single value is taken to represent a block of pixels.  

#Recurrent Neural Network

Recurrent neural networks (RNN) are not feedforward networks.  RNN have some of their neuron outputs feeding into neurons at lower levels in the network. For example, the Elman network depicted in the figure in http://www.willamette.edu/~gorr/classes/cs449/rnn1.html treats one of the hidden layers as though it were a part of the input layer.  This gives the recurrent network some memory so that the hidden neuron states depend on more than just the input presented to the network.  The states of the neurons also depend on the input history.  This makes them well suited for problems where that history is significant.  Examples include time series problems, speech recognition and handwriting recognition.  

#Stacked Auto Encoders and De-Noising Auto Encoders

Auto encoders are deep neural nets for unsupervised learning.  The first figure from 

http://ufldl.stanford.edu/wiki/index.php/Stacked_Autoencoders

depicts the first stage in the training process for a stacked auto encoder.  The network shown there has three layers - a 6 neuron input layer, a 4 neuron hidden layer and a temporary 6 neuron output layer.  This network gets trained using the inputs as the training targets.  Since the hidden layer has only 4 neurons, the training process is forced to develop a non-linear function of the inputs that best approximates the inputs but which has only 4 variables instead of the 6 that were present in the input.  The network is doing non-linear feature space reduction (also called manifold learning).  After this three layer network is trained the output layer is removed and 3 neuron hidden layer is added.  Then a (temporary) 4 neuron output layer is added and the output of the network is trained to reproduce the output of the first hidden layer. 

The resulting network can then have the temporary output layer removed and a real output layer added.  The whole network can then be trained against labels (supervised learning).  On the other hand the auto encoder stack can be used as a dimension reduction technique and the outputs (3 outputs in this simple example) can be used as inputs to another machine learning algorithm.  That architecture might make sense where some of the input data needs deep learning to unravel (extracting information from image data, for example) but other available inputs don't fit neatly alongside the data being processed in the auto encoder.  

#Two developments that make deep learning possible

Several recent advances have enabled the deep learning techniques currently in vogue.  To understand them, first take a look at Figure 2 from the Glorot, Bengio paper discussing training in deep neural nets. http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf.  

That figure shows the training behavior of a network with 4 hidden layers.  The network is classifying geometric shapes.  The input images are 32x32 pixels (1024 input neurons).  Figure 2 shows how the outputs of various layers behave as training progresses.  The network is using sigmoid activation functions so the outputs of all the hidden neurons fall between 0 and 1.  Notice the behavior of the outputs from the fourth hidden layer.  Those start around 0.5 - midrange for the sigmoid function.  Then very early in training they dive towards 0 - the far left side of the sigmoid function.  The problem with this behavior is that the training signal for all of the weights comes from the gradient of the output with respect to the weights, but for the sigmoid function the derivatives approach zero at the extremes of the function.  Very early in training the higher layers are driven into saturation and that causes the gradient to become very small.  That makes training take a long time.  

Notice that the 4th hidden layer is more severely saturated than the 3rd layer, which is also more serverely saturated than layers 1 and 2.  Layers 1 and 2 seem to behave reasonably.  So the problem is more severe for deep networks than for shallow ones.  This is the problem that has been overcome by modern advances in neural net training technology.  

One early approach to overcoming the training problem was to train and auto encoder stack as described above and to use the trained auto encoder stack as a starting point for supervised training.  That works well, but some papers have shown that better performance is possible.  

Some of the advances have been in the realm of the update algorithms used for updating weights using gradients.  You've studied several of those in lectures over the last two weeks.  Now you'll get to use them in training neural nets.  

Another important advance was understanding the relationship between the initial weight selections and subsequent training progress.  The Glorot, Bengio paper gives rules for selecting initial weights that in combination with update methods like AdaGrad and AdaDelt will enable you to train deep networks efficiently.  

#Q's

1.  Recall (or look up) the derivative of the sigmoid function.  What is its value for large arguments?  

2.  For what kinds of problems is deep learning a necessity?  Here are some for starters:
-Determine emotional state of customer from facial analysis
-Determine gender of customer from picture
-Convert spoken language to text. 
-Scan through some of the problems in UCI Data Repository. 
